In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import numpy as np
import matplotlib.pyplot as plt
from DSS_analyzer_Mariner import Data1D_GAUGE, Data1D_PumpingCurve
datapath = "./"
gauge_data_folder = datapath + "s_well/gauge_data/"
gauge_dataframe = Data1D_GAUGE.Data1D_GAUGE(gauge_data_folder + f"gauge6_data_swell.npz")
# Plot the gauge data
from DSS_analyzer_Mariner import Data1D_PumpingCurve
pumping_curve_filepath = datapath + f"prod/pumping_curve/pumping_curve_stage7.npz"
pumping_curve = Data1D_PumpingCurve.Data1D_PumpingCurve(pumping_curve_filepath)

# Crop the pressure gauge data,
start_time = pumping_curve.get_start_time()
end_time = pumping_curve.get_end_time()
gauge_dataframe.crop(start_time, end_time)

gauge_md_index = np.array([265, 520,  789, 1043, 1312, 1567])
frac_hit_index = np.array([589, 830])

In [17]:
# Interpolate the gauge data for any time point, the function would return the interpolated value
from scipy.interpolate import interp1d
def interpolate_gauge_data(time):
    gauge_data = gauge_dataframe.data
    gauge_taxis = gauge_dataframe.calculate_time()
    gauge_interpolator = interp1d(gauge_taxis * 3600, gauge_data, kind='linear')
    return gauge_interpolator(time)

In [18]:
# Simulator parameters
nx = 5000
# nx = 1800
total_time = 13000
d = 140 * np.ones(nx)
dx = 1
x = np.arange(0, nx*dx, dx)
x -= nx * dx / 2 # make the center of the domain as 0
u_snapshot = [] # store the snapshot of u
u_current = np.zeros(nx)
t = list([0]) # store the time points
dt = 0
dt_init = 10.0

In [19]:
# Load the pumping curve data at next stage
pumping_curve_filepath = datapath + f"prod/pumping_curve/pumping_curve_stage8.npz"
pumping_curve = Data1D_PumpingCurve.Data1D_PumpingCurve(pumping_curve_filepath)

In [20]:
end_time_full = pumping_curve.get_end_time()
end_time_full

datetime.datetime(2020, 3, 18, 14, 39, 26)

In [21]:
print("The total time of 2 stages is", (end_time_full - start_time).total_seconds(), "sec")

The total time of 2 stages is 36898.0 sec


In [21]:
# Simulator
def build_matrix_phase1(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
# Simulator
def build_matrix_phase1(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
    # Source term
    phase1_frac_hit_index = int(np.where(x == frac_hit_index[0])[0])
    A[phase1_frac_hit_index, :] = 0
    A[phase1_frac_hit_index, phase1_frac_hit_index] = 1
    
    return A

def build_rhs_phase1(u_current, t):
    rhs = u_current.copy()
    rhs[0] = 0
    rhs[-1] = 0
    # Source term.
    rhs[frac_hit_index[0]] = interpolate_gauge_data(t)
    # print(f"Time: {t}, Source: {interpolate_gauge_data(t)}") # debug info
    return rhs

In [ ]:
u_current = gauge_dataframe.data[0] * np.ones_like(u_current) # initial condition
u_snapshot.append(u_current.copy())

In [ ]:
# full step
def full_step_phase1(u_current, dt, t):
    A = build_matrix_phase1(dt)
    rhs = build_rhs_phase1(u_current, t)
    return np.linalg.solve(A, rhs)

# half step
def half_step_phase1(u_current, dt, t):
    dt_half = dt / 2
    A_half = build_matrix_phase1(dt_half)
    rhs_half = build_rhs_phase1(u_current, t)
    u_middle = np.linalg.solve(A_half, rhs_half)
    
    b_half = build_rhs_phase1(u_middle, t)
    u_final = np.linalg.solve(A_half, b_half)
    
    return u_final

In [ ]:
# Time sampling optimizer
def adjust_dt(dt_current, error, tol=1e-3):
    safety_factor = 0.9
    p = 2
    
    dt_new = safety_factor * dt_current * (tol / max(error, 1e-16))**(1/(p+1))
    
    dt_new = min(2.0 * dt_init, max(0.5 * dt_current, dt_new))
    
    return dt_new

In [ ]:
# test the full step
dt = dt_init

u_current = full_step_phase1(u_current, dt, t[-1])
u_snapshot.append(u_current.copy())
t.append(t[-1] + dt)

In [ ]:
while t[-1] < total_time: # time loop
    print("dt = ", dt)
    # calculate the full step
    u_current = full_step_phase1(u_current, dt, t[-1])
    
    # calculate the half step
    u_current_half = half_step_phase1(u_current, dt, t[-1])
    
    # calculate the error
    error = np.linalg.norm(u_current_half - u_current) / np.linalg.norm(u_current)
    # print(f"Time: {t[-1]}, Error: {error}")
    # adjust the time step
    dt = adjust_dt(dt, error)
    
    if error <= 1e-3:
        # store the snapshot if the error is small enough.
        u_snapshot.append(u_current.copy())
        t.append(t[-1] + dt)
        
        # update the time step
        dt = min(dt, 10)
    else:
        # the error is too large, reduce the time step. Don't store the snapshot.
        dt = max(0.5 * dt, 0.5)
    
    # store the snapshot
    u_snapshot.append(u_current.copy())
    print(f"Time: {t[-1]}, Error: {error:.6f}")
    t.append(t[-1] + dt)

# After stage 1 stimulation, apply the loop for different permeability values.

In [ ]:
total_time_interval = 3000

In [ ]:
# Simulator
def build_matrix_phase2(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
    # Source term: there is no source term in phase 2.
    return A

def build_rhs_phase2(u_current, t):
    rhs = u_current.copy()
    rhs[0] = 0
    rhs[-1] = 0
    # Source term. There is no source term in phase 2.
    return rhs

# full step
def full_step_phase2(u_current, dt, t):
    A = build_matrix_phase2(dt)
    rhs = build_rhs_phase2(u_current, t)
    return np.linalg.solve(A, rhs)

# half step
def half_step_phase2(u_current, dt, t):
    dt_half = dt / 2
    A_half = build_matrix_phase2(dt_half)
    rhs_half = build_rhs_phase2(u_current, t)
    u_middle = np.linalg.solve(A_half, rhs_half)
    
    b_half = build_rhs_phase2(u_middle, t)
    u_final = np.linalg.solve(A_half, b_half)
    
    return u_final

In [ ]:
# Add phase 3 simulator
# Simulator for phase 3. 
def build_matrix_phase3(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
# Simulator
def build_matrix_phase3(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
    # Source term
    phase3_frac_hit_index = int(np.where(x == frac_hit_index[1])[0])
    A[phase3_frac_hit_index, :] = 0
    A[phase3_frac_hit_index, phase3_frac_hit_index] = 1
    
    return A

def build_rhs_phase3(u_current, t):
    rhs = u_current.copy()
    rhs[0] = 0
    rhs[-1] = 0
    # Source term.
    rhs[frac_hit_index[1]] = interpolate_gauge_data(t)
    # print(f"Time: {t}, Source: {interpolate_gauge_data(t)}") # debug info
    return rhs

In [ ]:
# permeability values after the stage 1 stimulation
diff_change_ratio = np.array([0.5, 0.3, 0.1, 0.05, 0.01, 0.001])

In [ ]:
# In this case, I'll implement the first case: release the pressure, change the permeability. 
def pressure_diffusion_phase2(u_init, d):
    """
    A simplified wrapper function for performing a specific period of numerical simulation,
    mainly focusing on testing how different 'd' (diffusivity/permeability arrays) affect the results.
    
    Parameters
    ----------
    u_init : numpy.ndarray
        The initial field (e.g., pressure field).
    d : numpy.ndarray
        The diffusivity (or permeability) array related to the diffusion process.
        
    Returns
    ----------
    result_snapshots : numpy.ndarray
        An array of shape (N, ...) containing the solutions at various time steps.
    """
    
    # ========== 1. Preset internal parameters (modify if needed) ==========
    # Example index of the fracture hit
    frac_hit_index = (10,)
    
    # Total time interval for this simulation phase
    total_time_interval = 3000.0
    
    # Initial time step
    dt = 1.0
    
    # Time sequence, starting from 0
    t = [0.0]
    
    # Error threshold
    error_threshold = 1e-3
    
    # Max and min time step
    dt_max = 10.0
    dt_min = 0.5
    
    # ========== 2. Initialization ==========
    # Copy the initial field to avoid modifying the input directly
    u_current = u_init.copy()
    
    # Store snapshots of the solution
    u_snapshot = [u_current.copy()]
    
    # Calculate the end time for this simulation stage
    total_time = t[-1] + total_time_interval
    
    # ========== 3. Time-stepping loop ==========
    while t[-1] < total_time:
        print("dt =", dt)
        
        # Call the full-step and half-step functions (define them externally)
        u_full = full_step_phase2(u_current, dt, t[-1])
        u_half = half_step_phase2(u_current, dt, t[-1])
        
        # Compute the error
        error = np.linalg.norm(u_half - u_current) / np.linalg.norm(u_current)
        
        # Adjust the time step (define this function externally as well)
        dt_new = adjust_dt(dt, error)
        
        if error <= error_threshold:
            # Error is small enough, accept this step
            u_current = u_full.copy()
            u_snapshot.append(u_current.copy())
            t.append(t[-1] + dt)
            dt = min(dt_new, dt_max)
        else:
            # Error is too large, reduce dt and retry
            dt = max(0.5 * dt_new, dt_min)
        
        # Print current time step info
        print(f"Time: {t[-1]}, Error: {error:.6f}")
        
        # You could choose whether or not to store a snapshot here every iteration
        u_snapshot.append(u_current.copy())
        t.append(t[-1] + dt)

    # ========== 4. Convert snapshots to an array and return ==========
    result_snapshots = np.array(u_snapshot)
    return result_snapshots

In [ ]:
# Create phase 2 snapshot, (nx, nt, len(diff_change_ratio))
phase2_snapshots = []
for diff_ratio in diff_change_ratio:
    d_new = diff_ratio * d
    phase2_snapshot = pressure_diffusion_phase2(u_snapshot[-1], d_new)
    phase2_snapshots.append(phase2_snapshot)

# Phase 3

In [ ]:
# Add phase 3 simulator
total_time_stage2 = 36898 - total_time_interval - total_time
total_time += total_time_stage2

# Simulator for phase 3. 
def build_matrix_phase3(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
# Simulator
def build_matrix_phase3(dt):
    alpha = d * dt / dx**2
    
    A = np.zeros((nx, nx))
    
    for i in range(1, nx-1):
        A[i, i-1] = -alpha[i]
        A[i, i] = 1 + 2 * alpha[i]
        A[i, i+1] = -alpha[i]
    
    A[0, 0] = 1
    A[-1, -1] = 1
    A[0, 1] = -1
    A[-1, -2] = -1
    
    # Source term
    phase3_frac_hit_index = int(np.where(x == frac_hit_index[1])[0])
    A[phase3_frac_hit_index, :] = 0
    A[phase3_frac_hit_index, phase3_frac_hit_index] = 1
    
    return A

def build_rhs_phase3(u_current, t):
    rhs = u_current.copy()
    rhs[0] = 0
    rhs[-1] = 0
    # Source term.
    rhs[frac_hit_index[1]] = interpolate_gauge_data(t)
    # print(f"Time: {t}, Source: {interpolate_gauge_data(t)}") # debug info
    return rhs

In [ ]:
def pressure_diffusion_phase3(u_init, d):
    """
    A modified version of the Phase 2 simulation function, demonstrating how to
    release pressure and change permeability in a specific region (e.g., frac_hit_index).
    
    Parameters
    ----------
    u_init : numpy.ndarray
        The initial field (e.g., pressure field).
    d : numpy.ndarray
        The diffusivity (or permeability) array related to the diffusion process.
        
    Returns
    ----------
    result_snapshots : numpy.ndarray
        An array of shape (N, ...) containing the solutions at various time steps.
    """
    
    # ========== 1. Preset internal parameters (modify if needed) ==========
    frac_hit_index = (10,)       # Example index for changing permeability
    total_time_interval = 3000.0 # Longer total time to simulate pressure release
    dt = 1.0                     # Initial time step
    t = [0.0]                    # Start time
    error_threshold = 1e-3       # Error threshold
    dt_max = 10.0                # Max allowable time step
    dt_min = 0.5                 # Min allowable time step
    
    # ========== 2. Initialization ==========
    # Copy the initial field to avoid modifying the input directly
    u_current = u_init.copy()
    
    # Store snapshots of the solution
    u_snapshot = [u_current.copy()]
    
    # Calculate the end time for this simulation stage
    total_time = t[-1] + total_time_interval
    
    # Example: Change permeability (or diffusivity) at frac_hit_index
    # (simulates the effect of fracture hit, pressure release, etc.)
    d[frac_hit_index[0]] /= 200.0
    
    # ========== 3. Time-stepping loop ==========
    while t[-1] < total_time:
        print("dt =", dt)
        
        # Full-step update (implement or import externally)
        u_full = full_step_phase2(u_current, dt, t[-1])
        
        # Half-step update (implement or import externally)
        u_half = half_step_phase2(u_current, dt, t[-1])
        
        # Compute the error
        error = np.linalg.norm(u_half - u_current) / np.linalg.norm(u_current)
        
        # Adjust the time step (implement or import externally)
        dt_new = adjust_dt(dt, error)
        
        if error <= error_threshold:
            # Accept this step if the error is small enough
            u_current = u_full.copy()
            u_snapshot.append(u_current.copy())
            t.append(t[-1] + dt)
            dt = min(dt_new, dt_max)
        else:
            # If the error is too large, reduce dt, skip snapshot storage
            dt = max(0.5 * dt_new, dt_min)
        
        # Print current time step info
        print(f"Time: {t[-1]}, Error: {error:.6f}")
        
        # Optional: store additional snapshot every iteration
        u_snapshot.append(u_current.copy())
        t.append(t[-1] + dt)

    # ========== 4. Convert snapshots to an array and return ==========
    result_snapshots = np.array(u_snapshot)
    return result_snapshots

In [ ]:
# Create phase 3 snapshot, (nx, nt, len(diff_change_ratio))
phase3_snapshots = []
for diff_ratio in diff_change_ratio:
    d_new = diff_ratio * d
    phase3_snapshot = pressure_diffusion_phase3(phase2_snapshots[-1], d_new)
    phase3_snapshots.append(phase3_snapshot)

In [ ]:
# Save the snapshots
np.savez("phase2_snapshots.npz", phase2_snapshots)
np.savez("phase3_snapshots.npz", phase3_snapshots)
# phase 1 snapshots
np.savez("phase1_snapshots.npz", u_snapshot)